***Copyright 2024 by the authors' university which is unnamed because of KDD's anonymity requirement.***

##**Connect to Drive**

In [ ]:
from google.colab import drive
from google.colab import files

drive.mount('/content/gdrive/')
print("-"*80)

!ls "/content/gdrive/My Drive/Colab Notebooks/Research/CFE"

import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks/Research/CFE")
print("Current dir: ", os.getcwd())

##**Imports**

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings

from sklearn.svm import LinearSVC
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, precision_recall_curve, roc_curve, auc
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

# Suppress warnings
warnings.filterwarnings("ignore")

##**Linear SVM**

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/CFE_datasets/winequality-red.csv')
df = df.dropna()

df['quality'] = df['quality'].apply(lambda x: 0 if x <= 7 else 1)

X = df.loc[:, df.columns != 'quality']
y = df['quality']

rus = RandomOverSampler(random_state=42)
X, y = rus.fit_resample(X, y)
print(len(X), len(y))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

params = {
    'C': [1, 5, 10, 25, 50, 100]
}

model = LinearSVC(random_state=42)
cv = StratifiedKFold(n_splits=10)
grid_search = GridSearchCV(estimator=model, param_grid=params, n_jobs=-1, cv=cv, scoring='f1_macro')
grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)
rf_best = grid_search.best_estimator_
print("Best Estimator:", rf_best)

In [ ]:
best = LinearSVC(C=25, random_state=42)
best.fit(X, y)

cv = StratifiedKFold(n_splits=10)
cv_scores = cross_val_score(best, X, y, cv=cv, scoring='f1_macro')
print("Cross-Validation Scores:", cv_scores)
print("Mean CV Accuracy:", cv_scores.mean())

train_predictions = best.predict(X)
train_accuracy = accuracy_score(y, train_predictions)
print("Training Set Accuracy:", train_accuracy)

Cross-Validation Scores: [0.85190526 0.84148221 0.7900461  0.82660651 0.64401316 0.75223564
 0.48603246 0.81989904 0.92082185 0.79721925]
Mean CV Accuracy: 0.7730261486171932
Training Set Accuracy: 0.8418722327640734


##**Extracting weights and intercept values**

In [ ]:
for (intercept, coef) in zip(best.intercept_, best.coef_):
    s = "y = {0:.3f}".format(intercept)
    for (i, c) in enumerate(coef):
      s += " + {0} * x{1}".format(c, i)

    print(s)

y = 1.268 + -0.07741946495133757 * x0 + -0.45495224538578916 * x1 + 1.4579152719016355 * x2 + -0.051451136453482975 * x3 + -7.663862823783088 * x4 + 0.006493381810836352 * x5 + -0.0006536046170267842 * x6 + 1.1671895883897803 * x7 + -2.355864900290824 * x8 + 2.211876965207599 * x9 + 0.459289040884419 * x10


In [ ]:
b = best.intercept_
w = best.coef_

w = w.flatten()

##**Setting the original misclassified *x* feature vector**

In [ ]:
og_x = df.drop(columns=['quality'])
predicted_label = best.predict(og_x)

results_df = pd.DataFrame({'Quality_Predicted': predicted_label})
index_class = results_df.index[results_df['Quality_Predicted'] == 0]

og_x_reset = og_x.reset_index(drop=True)
filtered_records = og_x_reset.loc[index_class]
print(len(filtered_records))

1081


##**Z3 implementation**

In [ ]:
!pip install z3
!pip install z3-solver

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.3 MB/s eta 0:00:00
  Created wheel for z3: filename=z3-0.2.0-py3-none-any.whl size=26629 sha256=99175e117e3f646d55c4eeb6c30bb1207edfa3fe288f0ec7d5916d957688efa8
  Stored in directory: /root/.cache/pip/wheels/5a/b2/60/55b07a5084cad7ab411e395fb1440a2b1a19598bff535a3955
Successfully built z3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 10.5 MB/s eta 0:00:00


##**Optimizer -- step 1**

In [ ]:
from z3 import *
import csv
import time

def abs(x):
    return If(x < 0, -x, x)

def step1(W, B, og_x_np):
    n = W.shape[0]

    opt = Optimize()

    X = [Real(f'x_{i}') for i in range(n)]

    opt.add(And(X[0] >= 4, X[0] <= 6.7))

    opt.add(And(X[1] >= 0.2, X[1] <= 0.4))

    opt.add(And(X[2] >= 0.3, X[2] <= 0.55))

    opt.add(And(X[3] >= 6.7, X[3] <= 10))

    original_value4 = og_x_np[4]
    if original_value4 is not None:
        opt.add(X[4] == original_value4)

    original_value5 = og_x_np[5]
    if original_value5 is not None:
        opt.add(X[5] == original_value5)

    original_value6 = og_x_np[6]
    if original_value6 is not None:
        opt.add(X[6] == original_value6)

    original_value7 = og_x_np[7]
    if original_value7 is not None:
        opt.add(X[7] == original_value7)

    original_value8 = og_x_np[8]
    if original_value8 is not None:
        opt.add(X[8] == original_value8)

    original_value9 = og_x_np[9]
    if original_value9 is not None:
        opt.add(X[9] == original_value9)

    original_value10 = og_x_np[10]
    if original_value10 is not None:
        opt.add(X[10] == original_value10)

    dot_product = Sum([W[i] * X[i] for i in range(n)])

    inequality = dot_product + B > 0
    opt.add(inequality)

    l1_distance = Sum([abs(og_x_np[i] - X[i]) for i in range(n)])

    objective = l1_distance
    opt.minimize(objective)

    if opt.check() == sat:
        model = opt.model()

        original_f = [og_x_np[i] for i in range(n)]

        # Get the optimized feature vector x_prime
        x_prime_optimized = [model[X[i]].as_decimal(15) for i in range(n)]
        print(f"Optimized x': {x_prime_optimized}")

        # Calculate the optimized distance
        l1_distance_1 = model.eval(objective).as_decimal(15)
        print(f"Optimized distance: {l1_distance_1}")

        # Calculate the optimized cost
        cost_optimized = model.eval(objective).as_decimal(15)
        print(f"Optimized cost: {cost_optimized}")

        return original_f, x_prime_optimized, l1_distance_1, cost_optimized
    else:
        print("````````````FAILED TO FIND AN OPTIMAL SOLUTION.`````````````````````")
        original_f = [og_x_np[i] for i in range(n)]
        return [original_f, '', '', '']

W = w
B = b[0]
output_file = "/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/WineQuality/Step1.csv"

for i in range(len(filtered_records)):
    row = filtered_records.iloc[i]
    og_x_np = row.to_numpy()

    start_time = time.time()
    original_f, x_prime_optimized, l1_distance_1, cost_optimized = step1(W, B, og_x_np)
    end_time = time.time()
    runtime = end_time - start_time

    with open(output_file, 'a', newline='') as file:
        writer = csv.writer(file)
        data_row = [original_f] + [x_prime_optimized] + [l1_distance_1] + [cost_optimized] + [runtime]
        writer.writerow(data_row)

In [ ]:
column_names = ['Original_x', 'Optimal_x', 'Cost', 'Objective', 'Runtime']
df1 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/WineQuality/Step1.csv', names = column_names)

def remove_question_mark(cell_value):
    return cell_value.replace('?', '') if isinstance(cell_value, str) else cell_value

def remove_question_c(cell_value):
    return cell_value.replace('[', '') if isinstance(cell_value, str) else cell_value

def remove_question_cc(cell_value):
    return cell_value.replace(']', '') if isinstance(cell_value, str) else cell_value

def convert_to_float(lst_str):
    lst_str = str(lst_str)
    lst = lst_str.split(", ")
    return np.array([float(item.strip("'")) for item in lst])

cleaned_df = df1.applymap(remove_question_mark)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_c)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_cc)

# Apply the function to each row in the DataFrame
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].apply(convert_to_float)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_c)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_cc)

optimized = cleaned_df.dropna()
left = cleaned_df[cleaned_df.isna().any(axis=1)]

optimized.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/WineQuality/Step1_optimized.csv', index=False)
left.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/WineQuality/Step1_failed.csv', index=False)

for i in range(len(optimized['Optimal_x'])):
  x = optimized['Optimal_x'].iloc[i]
  predicted_label = best.predict(x.reshape(1, -1))
  print(predicted_label)

##**Optimizer -- step 2**

In [ ]:
def step2(W, B, row):
    n = W.shape[0]

    opt = Optimize()

    X = [Real(f'x_{i}') for i in range(n)]

    slack_fa = Real('slack_fa')
    slack_fa_constr = (slack_fa >= 0)
    opt.add(And(X[0] > 0, X[0] >= 4 - slack_fa, X[0] <= 6.7 + slack_fa))
    opt.add(slack_fa_constr)

    slack_va = Real('slack_va')
    slack_va_constr = (slack_va >= 0)
    opt.add(And(X[1] > 0, X[1] >= 0.2 - slack_va, X[1] <= 0.4 + slack_va))
    opt.add(slack_va_constr)

    slack_ca = Real('slack_ca')
    slack_ca_constr = (slack_ca >= 0)
    opt.add(And(X[2] > 0, X[2] >= 0.3 - slack_ca, X[2] <= 0.55 + slack_ca))
    opt.add(slack_ca_constr)

    slack_rs = Real('slack_rs')
    slack_rs_constr = (slack_rs >= 0)
    opt.add(And(X[3] > 0, X[3] >= 6.7 - slack_rs, X[3] <= 10 + slack_rs))
    opt.add(slack_rs_constr)

    original_value4 = row[4]
    if original_value4 is not None:
        opt.add(X[4] == original_value4)

    original_value5 = row[5]
    if original_value5 is not None:
        opt.add(X[5] == original_value5)

    original_value6 = row[6]
    if original_value6 is not None:
        opt.add(X[6] == original_value6)

    original_value7 = row[7]
    if original_value7 is not None:
        opt.add(X[7] == original_value7)

    original_value8 = row[8]
    if original_value8 is not None:
        opt.add(X[8] == original_value8)

    original_value9 = row[9]
    if original_value9 is not None:
        opt.add(X[9] == original_value9)

    original_value10 = row[10]
    if original_value10 is not None:
        opt.add(X[10] == original_value10)

    dot_product = Sum([W[i] * X[i] for i in range(n)])

    inequality = dot_product + B > 0
    opt.add(inequality)

    objective =  (slack_fa + slack_va + slack_ca + slack_rs)
    opt.minimize(objective)

    if opt.check() == sat:
        model = opt.model()

        original_f = [row[i] for i in range(n)]

        x_prime_optimized = [model[X[i]].as_decimal(15) for i in range(n)]
        print(f"Optimized x': {x_prime_optimized}")

        l1_distance_1 = model.eval(objective).as_decimal(15)
        print(f"Optimized distance: {l1_distance_1}")

        cost_optimized = model.eval(objective).as_decimal(15)
        print(f"Optimized cost: {cost_optimized}")

        slack_fa = model[slack_fa].as_decimal(15)
        print(f"slack_fa value: {slack_fa}")

        slack_va = model[slack_va].as_decimal(15)
        print(f"slack_va value: {slack_va}")

        slack_ca = model[slack_ca].as_decimal(15)
        print(f"slack_ca value: {slack_ca}")

        slack_rs = model[slack_rs].as_decimal(15)
        print(f"slack_rs value: {slack_rs}")

        return original_f, x_prime_optimized, l1_distance_1, cost_optimized, slack_fa, slack_va, slack_ca, slack_rs

    else:
        print("````````````FAILED TO FIND AN OPTIMAL SOLUTION.`````````````````````")
        original_f = [row[i] for i in range(n)]
        return [original_f, '', '', '', '', '', '', '']

W = w
B = b[0]
output_file1 = "/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/WineQuality/Step2.csv"
input_file = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/WineQuality/Step1_failed.csv')

input_file['Original_x'] = input_file['Original_x'].map(remove_question_c)
input_file['Original_x'] = input_file['Original_x'].map(remove_question_cc)

input_file['Original_x'] = input_file['Original_x'].apply(convert_to_float)

for i in range(len(input_file['Original_x'])):
    row = input_file['Original_x'].iloc[i]
    start_time = time.time()
    original_f, x_prime_optimized, cost, cost_optimized, slack_fa, slack_va, slack_ca, slack_rs = step2(W, B, row)
    end_time = time.time()
    runtime = end_time - start_time

    with open(output_file1, 'a', newline='') as file:
        writer = csv.writer(file)
        data_row = [original_f] + [x_prime_optimized] + [cost] + [cost_optimized] + [runtime] + [slack_fa] + [slack_va] + [slack_ca] + [slack_rs]
        writer.writerow(data_row)

Optimized x': ['4', '0.2', '1.206427060706219?', '6.7', '0.076', '11', '34', '0.9978', '3.51', '0.56', '9.4']
Optimized distance: 0.656427060706219?
Optimized cost: 0.656427060706219?
slack_fa value: 0
slack_va value: 0
slack_ca value: 0.656427060706219?
slack_rs value: 0
Optimized x': ['4', '0.2', '1.206427060706219?', '6.7', '0.076', '11', '34', '0.9978', '3.51', '0.56', '9.4']
Optimized distance: 0.656427060706219?
Optimized cost: 0.656427060706219?
slack_fa value: 0
slack_va value: 0
slack_ca value: 0.656427060706219?
slack_rs value: 0
Optimized x': ['4', '0.2', '1.194952457849857?', '6.7', '0.075', '13', '40', '0.9978', '3.51', '0.56', '9.4']
Optimized distance: 0.644952457849857?
Optimized cost: 0.644952457849857?
slack_fa value: 0
slack_va value: 0
slack_ca value: 0.644952457849857?
slack_rs value: 0
Optimized x': ['4', '0.2', '0.976516362080009?', '6.7', '0.069', '15', '59', '0.9964', '3.3', '0.46', '9.4']
Optimized distance: 0.426516362080009?
Optimized cost: 0.426516362080009

In [ ]:
# Read the CSV file into a DataFrame
column_names = ['Original_x', 'Optimal_x', 'Cost', 'Objective', 'Runtime', 'Slack_fa', 'Slack_va', 'Slack_ca', 'Slack_rs']
df_step2 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/New_implementation/WineQuality/Step2.csv', names = column_names)

df_step2 = df_step2.applymap(remove_question_mark)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_c)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_cc)

# Apply the function to each row in the DataFrame
df_step2['Optimal_x'] = df_step2['Optimal_x'].apply(convert_to_float)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_c)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_cc)

df_step2.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/WineQuality/Step2.csv', index=False)

for i in range(len(df_step2['Optimal_x'])):
  x = df_step2['Optimal_x'].iloc[i]
  predicted_label = best.predict(x.reshape(1, -1))
  print(predicted_label)

##**Optimizer -- step 3**

In [ ]:
def step3(W, B, row, slack_value1, slack_value2, slack_value3, slack_value4):
    n = W.shape[0]

    opt = Optimize()

    X = [Real(f'x_{i}') for i in range(n)]

    opt.add(And(X[0] >= 4 - slack_value1, X[0] <= 6.7 + slack_value1))

    opt.add(And(X[1] >= 0.2 - slack_value2, X[1] <= 0.4 + slack_value2))

    opt.add(And(X[2] >= 0.3 - slack_value3, X[2] <= 0.55 + slack_value3))

    opt.add(And(X[3] >= 6.7 - slack_value4, X[3] <= 10 + slack_value4))

    original_value4 = row[4]
    if original_value4 is not None:
        opt.add(X[4] == original_value4)

    original_value5 = row[5]
    if original_value5 is not None:
        opt.add(X[5] == original_value5)

    original_value6 = row[6]
    if original_value6 is not None:
        opt.add(X[6] == original_value6)

    original_value7 = row[7]
    if original_value7 is not None:
        opt.add(X[7] == original_value7)

    original_value8 = row[8]
    if original_value8 is not None:
        opt.add(X[8] == original_value8)

    original_value9 = row[9]
    if original_value9 is not None:
        opt.add(X[9] == original_value9)

    original_value10 = row[10]
    if original_value10 is not None:
        opt.add(X[10] == original_value10)

    dot_product = Sum([W[i] * X[i] for i in range(n)])

    inequality = dot_product + B > 0
    opt.add(inequality)

    l0 = Sum([If(row[i] != X[i], 1, 0) for i in range(n)])

    objective = l0
    opt.minimize(objective)

    if opt.check() == sat:
        model = opt.model()

        original_f = [row[i] for i in range(n)]

        # Get the optimized feature vector x_prime
        x_prime_optimized = [model[X[i]].as_decimal(15) for i in range(n)]
        print(f"Optimized x': {x_prime_optimized}")

        # Calculate the optimized distance
        l0_distance_1 = model.eval(objective)
        print(f"Distance: {l0_distance_1}")

        # Calculate the optimized distance
        cost_optimized = model.eval(objective)
        print(f"Objective: {cost_optimized}")

        return original_f, x_prime_optimized, l0_distance_1, cost_optimized
    else:
        print("````````````FAILED TO FIND AN OPTIMAL SOLUTION.`````````````````````")

        original_f = [row[i] for i in range(n)]
        return [original_f, '', '', '']

W = w
B = b[0]
out = '/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/WineQuality/Step3.csv'
last = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/WineQuality/Step2.csv')

last['Original_x'] = last['Original_x'].map(remove_question_c)
last['Original_x'] = last['Original_x'].map(remove_question_cc)
last['Original_x'] = last['Original_x'].apply(convert_to_float)

for i in range(len(last['Original_x'])):
    row = last['Original_x'].iloc[i]
    slack_value1 = last['Slack_fa'].iloc[i]
    slack_value2 = last['Slack_va'].iloc[i]
    slack_value3 = last['Slack_ca'].iloc[i]
    slack_value4 = last['Slack_rs'].iloc[i]

    start_time = time.time()
    original_f, x_prime_optimized, l1_distance_1, cost_optimized = step3(W, B, row, slack_value1, slack_value2, slack_value3, slack_value4)
    end_time = time.time()
    runtime = end_time - start_time

    with open(out, 'a', newline='') as file:
        writer = csv.writer(file)
        data_row = [original_f] + [x_prime_optimized] + [l1_distance_1] + [cost_optimized] + [runtime]
        writer.writerow(data_row)

In [ ]:
# Read the CSV file into a DataFrame
column_names = ['Original_x', 'Optimal_x', 'Cost', 'Objective', 'Runtime']
df_step3 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/WineQuality/Step3.csv', names = column_names)

df_step3 = df_step3.applymap(remove_question_mark)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_c)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_cc)

# Apply the function to each row in the DataFrame
df_step3['Optimal_x'] = df_step3['Optimal_x'].apply(convert_to_float)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_c)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_cc)

df_step3.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/WineQuality/Step3.csv', index=False)

for i in range(len(df_step3['Optimal_x'])):
  x = df_step3['Optimal_x'].iloc[i]
  predicted_label = best.predict(x.reshape(1, -1))
  print(predicted_label)

## **Average runtimes for l0 & l1**

In [ ]:
final_l0 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/WineQuality/Step3.csv')
l0_runtime = final_l0.loc[:, 'Runtime'].mean()

print('L0 runtime', l0_runtime)

L0 0.02056769366319328


In [ ]:
d3_prior = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/WineQuality/Step2.csv')
d3_initial = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/WineQuality/Step1_failed.csv')
d1 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/WineQuality/Step1_optimized.csv')

d = pd.concat([d1, d3_initial])
l1_runtime = d.loc[:, 'Runtime'].mean()
print('L1 runtime', l1_runtime)

0.04930534239282874


## **Percentage of relaxations**

In [ ]:
relaxed_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/WineQuality/Step2.csv')
needed_relaxing = len(relaxed_df)

subset_of_experiment = len(filtered_records)

percentage_of_relaxations = (needed_relaxing/subset_of_experiment)*100
print(percentage_of_relaxations)

56.151711378353376


### **Average percentage of relaxed soft constraints**

In [ ]:
relaxed_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/WineQuality/Step2.csv')
number_of_slacks_per_record = 4

count_s1 = 0
count_s2 = 0
count_s3 = 0
count_s4 = 0
total_count = 0
total_sum = 0
for i in range(len(relaxed_df)):
    if relaxed_df['Slack_fa'].iloc[i] != 0:
        count_s1 = 1
    elif relaxed_df['Slack_fa'].iloc[i] == 0:
        count_s1 = 0
    if relaxed_df['Slack_va'].iloc[i] != 0:
        count_s2 = 1
    elif relaxed_df['Slack_va'].iloc[i] == 0:
        count_s2 = 0
    if relaxed_df['Slack_ca'].iloc[i] != 0:
        count_s3 = 1
    elif relaxed_df['Slack_ca'].iloc[i] == 0:
        count_s3 = 0
    if relaxed_df['Slack_rs'].iloc[i] != 0:
        count_s4 = 1
    elif relaxed_df['Slack_rs'].iloc[i] == 0:
        count_s4 = 0
    total_count = count_s1 + count_s2 + count_s3 + count_s4
    total_sum += (total_count / number_of_slacks_per_record)

average_per_dataset = total_sum / len(relaxed_df) * 100

print(average_per_dataset)

25.0
